In [1]:
import os
import cv2
import numpy as np
from keras.preprocessing import image
from keras.preprocessing.image import img_to_array,load_img
from keras.models import load_model
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.applications.mobilenet import MobileNet,preprocess_input
from keras.models import Model
from keras.layers import Flatten,Dense
from keras.preprocessing.image import ImageDataGenerator


Using TensorFlow backend.


In [2]:
def create_model():
    base_model=MobileNet(input_shape=(224,224,3),include_top=False)
    for layer in base_model.layers:
        layer.trainable=False        
    x=Flatten()(base_model.output)
    x=Dense(units=7,activation='softmax')(x)
    model=Model(base_model.input,x)
    return model    

In [3]:
model=create_model()
model.load_weights('model.h5')

In [4]:
face_haar_cascade=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [5]:
cap=cv2.VideoCapture(0)

In [ ]:
while True:
  ret,test_img=cap.read()
  if not ret:
    continue
  gray_img=cv2.cvtColor(test_img,cv2.COLOR_BGR2RGB)
  faces_detected=face_haar_cascade.detectMultiScale(gray_img,1.3,5)
  for (x,y,w,h) in faces_detected:
    cv2.rectangle(test_img,(x,y),(x+w,y+h),(255,0,0),thickness=7)
    roi_gray=gray_img[y:y+w,x:x+h]
    roi_gray=cv2.resize(roi_gray,(224,224))
    img_pixels=image.img_to_array(roi_gray)
    img_pixels=np.expand_dims(img_pixels,axis=0)
    img_pixels/=255
    predictions=model.predict(img_pixels)
    max_index=np.argmax(predictions[0])
    emotions=("angry","disgust","fear","happy","sad","surprise","neutral")
    predicted_emotion=emotions[max_index]
    cv2.putText(test_img,predicted_emotion,(int(x),int(y)),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),2)
  resized_img=cv2.resize(test_img,(1000,700))
  cv2.imshow('Facial emotion analysis',resized_img)
  if cv2.waitKey(10)==ord('q'):
    break
  cap.release()
  cv2.destroyAllWindows()